In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit

In [2]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix
        ######
        ## имеет смысл сделать данные для одной группы и если групп несколько то сделать коллекцию из таких классов
        
        self.sig_s = [sum(matrix) for matrix in self.sig_scattering_matrix]
            
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        

In [3]:
class Sphere:
    # x, y, z -- как вектор, сделать операции векторными
    def __init__(self, x_0, y_0, z_0, r):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 =z_0
        self.r = r
        # класс должен говорить на языке геометри -- никаких частиц нет, есть точка и направление
        
    def distance(self, particle):
        x_ = particle.coordinates.x - self.x_0
        y_ = particle.coordinates.y - self.y_0
        z_ = particle.coordinates.z - self.z_0
        
        c = x_ * x_ + y_ * y_ + z_ * z_ - self.r * self.r
        
        k = x_ * particle.direction.tetta_x + y_ * particle.direction.tetta_y + z_ * particle.direction.tetta_z
        
        dec = k * k - c
        #заменить на бесконечность
        if dec < 0:
            return -1
        
        dist_1 = (-k + math.sqrt(k * k - c))
        dist_2 = (-k - math.sqrt(k * k - c))
         
        distance = min(dist_1, dist_2)
        
        return distance
     #вообще - то это не ответсвенность сферы, но мы пока переименуем angle  в направление
    def get_reflected_direction(self, angle):
        
        eps = random.uniform(0, math.pi * 2)
        tetta = random.uniform(0, math.pi)
        r_reflected = Direction(eps, tetta)
        
        return r_reflected  
    
    def get_sign(self, particle):
            
        sign = ((self.x_0 - particle.coordinates.x) * (self.x_0 - particle.coordinates.x) + 
        (self.y_0 - particle.coordinates.y) * (self.y_0 - particle.coordinates.y)  + 
                (self.z_0 - particle.coordinates.z) * (self.z_0 - particle.coordinates.z) - self.r * self.r)
        
        if sign == 0:
            return 0
            
        if sign < 0:
            return -1
        else:
            return 1  
        
    def get_xml(self):

        obj_xml = " Sphere with x_0 "+ str(x_0) + " y_0 "+ str(y_0) + " z_0 " + str(z_0) + " r = "+ str(r)
        
        return obj_xml

In [4]:
energy_groups_2 = [0, 6.25, 2 * 1E10]


# сделать так чтобы класс конвертировался в нумпай аррау (можно сделать арифметические операции)
class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [5]:
# проверить на геттеры и сеттеры

class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        #  если объект для одной группы, то проще всего хранить этот объект
        # убрать зависимость от индексов
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def get_energy_group(self): 
        
        return self.energy_group
     #   res = next(x for x, val in enumerate(self.energy_groups)
      #                            if val > self.energy)        
       # return res - 1
    
    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [6]:
# сделать векторный класс
# методы приватные и они не меняются

class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        # проверить случай что частица в плоскости
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        signed_distance = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        return np.sign(signed_distance)

    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [7]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [8]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [9]:
import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [10]:
def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [11]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
         #   particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
          #  particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [12]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [13]:
def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False

In [14]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [15]:
def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx)   
    set_random_direction(particle)

In [16]:
def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    

In [17]:
def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        process_scattering(particle, c_s)
        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        

In [18]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [19]:
def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    
    while not particle.is_terminated():
        set_random_direction(particle)
        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions

In [20]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [42]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])       

In [84]:
class Flux_group_universe_estimator:
    def __init__(self, universe, group):
        self.boundaries = universe
        self.group = group
        self.collision_sum = [0]
        
    def add_collision(self, particle):
        if particle.get_energy_group() == self.group and self.boundaries.is_inside(particle):
            self.collision_sum[-1]  += particle.get_weight()    

In [123]:
def get_sphere_estimators(radius, epsilon, group_number):
    
    estimators = []
    
    points_to_divide = [0.241394, 0.502905, 0.744300]
    
    test_sphere = Sphere(0, 0, 0, 0.8)
    boundaries_type = ["black"]

    surfaces = [test_sphere]
    signs = [-1]
    
    sphere = Cell(surfaces, signs)
    estimators.append(Flux_group_universe_estimator(sphere, group_number))
    
    
    for point in points_to_divide:
        
        test_sphere_inside = Sphere(0, 0, 0, epsilon + radius * point)

        test_sphere_outside = Sphere(0, 0, 0, radius * point - epsilon)

        surfaces = [test_sphere_inside, test_sphere_outside]
        signs = [-1, +1]

        sphere = Cell(surfaces, signs)
        estimators.append(Flux_group_universe_estimator(sphere, group_number)) 
    
    
    test_sphere_inside = Sphere(0, 0, 0, radius)
    test_sphere_outside = Sphere(0, 0, 0, radius - epsilon)

    
    surfaces = [test_sphere_inside, test_sphere_outside]
    signs = [-1, +1]
    
    
    sphere = Cell(surfaces, signs)
    estimators.append(Flux_group_universe_estimator(sphere, group_number))
    
    
    return estimators

In [124]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [125]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [126]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [127]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [128]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [129]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   

In [130]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [131]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    box_size = [0.2, 0.2, 0.2]
    zero_point = [0., 0., 0.]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [132]:
cs_fission_pu_239 = [0.08544, 0.0936]

cs_capture_pu_239 = [0.0144, 0.00480]

cs_scattering_pu_239 = [[0.23616, 0.0], [0.0432, 0.0792]]

cs_total_pu_239 = [0.3360, 0.2208]

cs_production_neutrons_pu_239 = [2.93, 3.10]

cs_xi_table_pu_239 = [ 0.425, 0.575]


pu_239 = Sig(2, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]

In [133]:
cs_fission_pu_239 = [0.050632, 0.0010484]

cs_capture_pu_239 = [0.025788, 0.0010046]

cs_scattering_pu_239 = [[2.44383, 0.0], [0.029227, 0.62568]]

cs_total_pu_239 = [2.52025, 0.65696]

cs_production_neutrons_pu_239 = [2.5, 2.5]

cs_xi_table_pu_239 = [0., 1.]


pu_239 = Sig(2, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                       cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]

In [183]:
test_sphere = Sphere(0, 0, 0, 16.049836)
boundaries_type = ["black"]

surfaces = [test_sphere]
signs = [-1]

sphere = Cell(surfaces, signs)
sphere.set_boundaries_type(boundaries_type)

In [201]:
cells = [sphere]

In [205]:
estimators_zero_group = get_sphere_estimators(16.049836, 0.1, 0)

In [206]:
estimators_first_group = get_sphere_estimators(16.049836, 0.1, 1)

In [207]:
estimators = []

In [208]:
estimators.extend(estimators_zero_group)

In [209]:
estimators.extend(estimators_first_group)

In [210]:
materials = [pu_239]

In [211]:
U235_Reactor = Universe(cells, materials)

In [212]:
test_number_of_particles = 7
test_number_interations = 200
test_number_inactive = 100

In [213]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, pu_239, estimators)

 num1  3
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
 keff_cycle , k_effective_exp, std_k_effective 1.0278273045071942   1.0008564640344546  0.02697084047273951
i == 103
 keff_cycle , k_effective_exp, std_k_effective 1.03782907

 keff_cycle , k_effective_exp, std_k_effective 1.033882023833463   1.000666175010925  0.007553209363178768
i == 166
 keff_cycle , k_effective_exp, std_k_effective 0.9504034796594132   0.9999046190207505  0.007476772156915311
i == 167
 keff_cycle , k_effective_exp, std_k_effective 0.9648558083085256   0.9993815024429561  0.007382889189475633
i == 168
 keff_cycle , k_effective_exp, std_k_effective 0.8938781122585142   0.9978299819990673  0.007437144603985113
i == 169
 keff_cycle , k_effective_exp, std_k_effective 0.9378646117609436   0.9969609186622829  0.007379916671508122
i == 170
 keff_cycle , k_effective_exp, std_k_effective 0.9951139179702945   0.9969345329381117  0.007273773141305336
i == 171
 keff_cycle , k_effective_exp, std_k_effective 0.9489851035242143   0.9962591888618596  0.007202326391751185
i == 172
 keff_cycle , k_effective_exp, std_k_effective 0.8611715865205918   0.9943829721626753  0.007345254512171584
i == 173
 keff_cycle , k_effective_exp, std_k_effective 0.852174262

In [214]:
def divide_estimators_square(average_number_of_collision, radius, epsilon, zenter):
    
    ratios_divide = [0.241394, 0.502905, 0.744300]
    
    average_number_of_collision[0] /= np.pi * 4/3 * zenter * zenter * zenter
    
    radius_plus_eps = ratios_divide[0] * radius + epsilon
    radius_minus_eps = ratios_divide[0] * radius - epsilon
    
    average_number_of_collision[1] /= np.pi * 4/3 * ((radius_plus_eps) * (radius_plus_eps) * (radius_plus_eps) -
                                (radius_minus_eps) * (radius_minus_eps) * (radius_minus_eps))
    
    radius_plus_eps = ratios_divide[1] * radius + epsilon
    radius_minus_eps = ratios_divide[1] * radius - epsilon
    
    average_number_of_collision[2] /= np.pi * 4/3 * ((radius_plus_eps) * (radius_plus_eps) * (radius_plus_eps) -
                                (radius_minus_eps) * (radius_minus_eps) * (radius_minus_eps))
    
    
    radius_plus_eps = ratios_divide[2] * radius + epsilon
    radius_minus_eps = ratios_divide[2] * radius - epsilon
    
    average_number_of_collision[3] /= np.pi * 4/3 * ((radius_plus_eps) * (radius_plus_eps) * (radius_plus_eps) -
                                (radius_minus_eps) * (radius_minus_eps) * (radius_minus_eps))
    
    average_number_of_collision[4] /= np.pi * 4/3 * ((radius) * (radius) * (radius) -
                                            (radius - epsilon) * (radius - epsilon) * (radius - epsilon))
    
    
    
    result_estimators = [ average_number_of_collision[1]/average_number_of_collision[0], 
                         average_number_of_collision[2]/ average_number_of_collision[0],
                        average_number_of_collision[3]/ average_number_of_collision[0],
                        average_number_of_collision[4]/ average_number_of_collision[0]]
    
    return result_estimators   

In [215]:
def get_std_flux(estimators):
    
    current_std = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[100:len(current_estimator.collision_sum) - 1]
        current_std.append(get_std(array_without_zeros))
    
    return current_std  

In [216]:
def get_average_number_of_collision(estimators):
    
    average_number_of_collision = []
    for i in range(0, len(estimators)):
        current_estimator = estimators[i]
        array_without_zeros = current_estimator.collision_sum[100:len(current_estimator.collision_sum) - 1]
        average_number_of_collision.append((sum(array_without_zeros)) / len(array_without_zeros))        
            
    return average_number_of_collision


In [217]:
average_sum_of_collision = get_average_number_of_collision(estimators_result)

In [218]:
print(average_sum_of_collision)

[2.56019946212595, 38.19125389446862, 117.26626218672267, 139.46478029919348, 8.601894070058709, 1.9080955990990163, 27.224204013295957, 84.84866542290231, 101.31431179095502, 14.048386367948764]


In [219]:
flux_sphere = divide_estimators_square(average_sum_of_collision, 16.049836, 0.1, 0.8)

In [220]:
print(flux_sphere)

[0.8478484947163983, 0.5999052548445266, 0.3257332370571344, 0.022399406860904598]


In [221]:
flux_std = get_std_flux(estimators_result)

print(flux_std)

[0.472862172492701, 1.6144944997972708, 2.5199537105049283, 3.1645880696131097, 0.44289717910812537, 0.19991228746387635, 0.8237511735079708, 1.4310173736980278, 1.366845899756442, 0.3981352207637061]


In [222]:


#test_flux_0 = [0.340124, 0.273056, 0.173845, 0.0212324]

test_flux = [0.943363, 0.761973, 0.504012, 0.147598]

for i in range(0, len(test_flux)):
    print(str(((flux_sphere[i] - test_flux[i])/test_flux[i] )* 100) + " % " + str(flux_std[i + 1] * 100000))



-10.124894158834046 % 161449.44997972707
-21.269486603261978 % 251995.37105049283
-35.37192823640421 % 316458.80696131097
-84.82404445798412 % 44289.717910812535


In [153]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  60.090750355057395  with standart deviation [pcm]+- 142.460091652966


In [39]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [40]:
file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)

In [223]:
def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()


In [224]:
file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)